##### Produce driver standings

In [0]:
# Create a text widget for file date with a default value
dbutils.widgets.text("p_file_date", "2021-03-28")

# Retrieve the value of the file date widget
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
# Running the notebook that contains common functions used across multiple notebooks
%run "../includes/common_functions"

In [0]:
# Load configuration settings from the specified notebook
%run "../includes/configuration"

Find race years for which the data is to be reprocessed

In [0]:
# Load the race results data from the Delta table and filter by the specified file date
race_results_df = spark.read.format("delta").load(f"{presentation_folder_path}/race_results") \
.filter(f"file_date = '{v_file_date}'")

In [0]:
# Convert the 'race_year' column of the race_results_df DataFrame to a list
race_year_list = df_column_to_list(race_results_df, 'race_year')

In [0]:
from pyspark.sql.functions import col

# Load the race results data from the Delta table and filter by the race years in race_year_list
race_results_df = spark.read.format("delta").load(f"{presentation_folder_path}/race_results") \
.filter(col("race_year").isin(race_year_list))

In [0]:
from pyspark.sql.functions import sum, when, count, col

# Aggregate race results to compute total points and wins for each driver per year
driver_standings_df = race_results_df \
.groupBy("race_year", "driver_name", "driver_nationality") \
.agg(sum("points").alias("total_points"),  # Sum of points for each driver
     count(when(col("position") == 1, True)).alias("wins"))  # Count of wins for each driver

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, rank

# Define window specification to rank drivers by total points and wins within each race year
driver_rank_spec = Window.partitionBy("race_year").orderBy(desc("total_points"), desc("wins"))

# Add a rank column to the driver standings DataFrame based on the window specification
final_df = driver_standings_df.withColumn("rank", rank().over(driver_rank_spec))

In [0]:
# Define the condition for merging the target and source tables
merge_condition = "tgt.driver_name = src.driver_name AND tgt.race_year = src.race_year"

# Merge the final DataFrame into the Delta table in the specified folder path
merge_delta_data(final_df, 'f1_presentation', 'driver_standings', presentation_folder_path, merge_condition, 'race_year')

In [0]:
%sql
-- Select all columns from the driver_standings table for the year 2021
SELECT * 
  FROM f1_presentation.driver_standings 
 WHERE race_year = 2021;

In [0]:
%sql
-- Count the number of records for each race year in the driver_standings table
SELECT race_year, COUNT(1)
  FROM f1_presentation.driver_standings
 GROUP BY race_year
 ORDER BY race_year DESC;